# Preparations

In [167]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import pickle
import os
import geojson
from geopy.geocoders import Nominatim
from shapely import geometry

In [26]:
request_cache = {}
if os.path.exists("reqests.cache.pkl"):
    with open("requests.cache.pkl", "rb") as f:
        request_cache = pickle.load(f)

In [39]:
def request_url(url):
    if url not in request_cache:
        print("Requesting", url)
        request_cache[url] = requests.get(url)
        with open("requests.cache.pkl", "wb") as f:
            pickle.dump(request_cache, f)
    else:
        print("Using cached", url)
    return request_cache[url]

# Numbeo scraping functions

In [124]:
def extract_numbeo_table(soup: BeautifulSoup, datacol: int) -> dict:
    table = soup.find("table", {"class": "data_wide_table"})
    table_rows = table.find_all("tr")
    table_rows = filter(lambda x: not x.find("th"), table_rows)
    table_rows = map(lambda x: x.findChildren("td"), table_rows)
    return {
        row[0].text.strip(): row[datacol].text for row in table_rows
    }

In [127]:
def parse_numbeo_eur(eur: str) -> float:
    # inp e,g. ' 445.30\xa0€'
    return float(eur.split()[0].replace(",", ""))

def parse_numbeo_satisfaction(sat: str) -> float:
    # inp. e.g. '80.36\nVery High'
    return float(sat.split()[0])/100

def assert_exists(page: requests.Response):
    if page.status_code == 404:
        raise ValueError("Page not found")
    soup = BeautifulSoup(page.content, "html.parser")
    # find div with style "error_message"
    if soup.find("div", {"style": "error_message"}):
        raise ValueError("Page not found")

In [102]:
def numbeo_col(city: str) -> dict:
    url = f"https://www.numbeo.com/cost-of-living/in/{city}"
    page = request_url(url)
    assert_exists(page)
    soup = BeautifulSoup(page.content, "html.parser")
    return extract_numbeo_table(soup, 1)

def numbeo_polution(city: str) -> dict:
    url = f"https://www.numbeo.com/pollution/in/{city}"
    page = request_url(url)
    assert_exists(page)
    soup = BeautifulSoup(page.content, "html.parser")
    res = extract_numbeo_table(soup, 2)
    return {
        k: parse_numbeo_satisfaction(v)
        for k, v in res.items()
    }

def numbeo_hc(city: str) -> dict:
    url = f"https://www.numbeo.com/health-care/in/{city}"
    page = request_url(url)
    assert_exists(page)
    soup = BeautifulSoup(page.content, "html.parser")
    res = extract_numbeo_table(soup, 2)
    return {
        k: parse_numbeo_satisfaction(v)
        for k, v in res.items()
    }

In [103]:
def numbeo(city: str) -> dict:
    try:
        return {
            "col": numbeo_col(city),
            "polution": numbeo_polution(city),
            "hc": numbeo_hc(city),
        }
    except ValueError:
        print("Err: not found", city)
        return None

# Life Expectancy

In [234]:
nuts_shapes = geojson.load(open("ref-nuts-2021-10m.geojson/NUTS_RG_10M_2021_4326.geojson", "rb"))

In [251]:
df_liveexp = pd.read_csv("NUTS 2 life expectancy 2021.csv")
df_liveexp = df_liveexp[df_liveexp["TIME_PERIOD"] == 2021]
df_liveexp = df_liveexp[df_liveexp["sex"] == "T"]
df_liveexp = df_liveexp.set_index("geo")["OBS_VALUE"]
df_liveexp.head()

geo
AL      75.5
AT      81.3
AT1     80.5
AT11    81.4
AT12    80.7
Name: OBS_VALUE, dtype: float64

In [207]:
def geocoords(city: str):
    geolocator = Nominatim(user_agent="my-app")
    loc = geolocator.geocode(city)
    return (loc.latitude, loc.longitude)

In [222]:
def nuts(city: str):
    (lat, lon) = geocoords(city)
    pt = geometry.Point(lon, lat)
    for feature in nuts_shapes["features"]:
        s = geometry.shape(feature["geometry"])
        if s.contains(pt):
            yield feature["properties"]

def nutsl(city: str, level: int):
    return filter(lambda x: x["LEVL_CODE"] == level, nuts(city))

In [281]:
def life_expectancy(city: str) -> float:
    regions = list(nuts(city))
    regions.sort(key=lambda x: x["LEVL_CODE"], reverse=True)
    for region in regions:
        if region["NUTS_ID"] in df_liveexp.index:
            return df_liveexp.loc[region["NUTS_ID"]]

# Scrape data

In [267]:
cities_nl = "Amsterdam Delft Rotterdam Utrecht Nijmegen Enschede Eindhoven".split()
cities_nl = [("nl", city) for city in cities_nl]
cities_de = "Aachen Cologne Dusseldorf Bonn Berlin".split()
cities_de = [("de", city) for city in cities_de]
cities_be = "Antwerp Leuven Brussels".split()
cities_be = [("be", city) for city in cities_be]
cities_other = [("ch", "Zurich"), ("dk", "Copenhagen"), ("no", "Oslo"), ("lu", "Luxembourg")]
cities = cities_nl + cities_de + cities_be + cities_other

In [268]:
errs = []
dataset = []
for country, city in cities:
    data = numbeo(city)
    if data is None:
        errs.append(city)
    else:
        dataset.append((country, city, data))

Using cached https://www.numbeo.com/cost-of-living/in/Amsterdam
Using cached https://www.numbeo.com/pollution/in/Amsterdam
Using cached https://www.numbeo.com/health-care/in/Amsterdam
Using cached https://www.numbeo.com/cost-of-living/in/Delft
Using cached https://www.numbeo.com/pollution/in/Delft
Using cached https://www.numbeo.com/health-care/in/Delft
Using cached https://www.numbeo.com/cost-of-living/in/Rotterdam
Using cached https://www.numbeo.com/pollution/in/Rotterdam
Using cached https://www.numbeo.com/health-care/in/Rotterdam
Using cached https://www.numbeo.com/cost-of-living/in/Utrecht
Using cached https://www.numbeo.com/pollution/in/Utrecht
Using cached https://www.numbeo.com/health-care/in/Utrecht
Using cached https://www.numbeo.com/cost-of-living/in/Nijmegen
Using cached https://www.numbeo.com/pollution/in/Nijmegen
Using cached https://www.numbeo.com/health-care/in/Nijmegen
Using cached https://www.numbeo.com/cost-of-living/in/Enschede
Using cached https://www.numbeo.com/po

In [269]:
df = pd.DataFrame(columns=["country", "city", "apartment centre", "apartment outside", "net salary", "air quality", "city quality", "greenness"])
for country, city, data in dataset:
    rent_inside = parse_numbeo_eur(data["col"]["Apartment (1 bedroom) in City Centre"])
    rent_outside = parse_numbeo_eur(data["col"]["Apartment (1 bedroom) Outside of Centre"])
    salary_net = parse_numbeo_eur(data["col"]["Average Monthly Net Salary (After Tax)"])
    df.loc[len(df)] = [
        country, city,
        rent_inside, rent_outside, salary_net,
        1-data["polution"]["Air Pollution"],
        1-data["polution"]["Dissatisfaction to Spend Time in the City"],
        1-data["polution"]["Dissatisfaction with Green and Parks in the City"],
    ]

In [288]:
df["life_expectancy"] = df["city"].apply(life_expectancy)

In [291]:
# sort by apartment centre / net salary
df["apartment centre / net salary"] = df["apartment centre"] / df["net salary"]
df = df.sort_values(by="apartment centre / net salary", ascending=True)
df

,country,city,apartment centre,apartment outside,net salary,air quality,city quality,greenness,life_expectancy,apartment centre / net salary
7,de,Aachen,586.14,445.30,2790.88,0.6900,0.6957,0.6905,81.2,0.210020
10,de,Bonn,625.00,586.67,2142.75,0.8088,0.8571,0.8864,81.2,0.291681
5,nl,Enschede,965.56,808.00,3041.25,0.8500,0.9444,0.8214,81.1,0.317488
9,de,Dusseldorf,999.74,723.25,3092.33,0.6532,0.7750,0.7768,80.4,0.323297
13,be,Leuven,814.11,705.71,2456.19,0.5500,0.8250,0.8500,83.6,0.331452
15,ch,Zurich,2204.53,1642.14,6323.78,0.8211,0.8533,0.7412,83.8,0.348610
6,nl,Eindhoven,1213.03,967.42,3453.86,0.7697,0.8542,0.8500,81.6,0.351210
14,be,Brussels,950.48,798.73,2700.15,0.3621,0.5553,0.7054,81.4,0.352010
12,be,Antwerp,824.12,678.46,2306.57,0.3704,0.5625,0.5650,82.9,0.357292
17,no,Oslo,13838.26,11882.86,38602.14,0.7708,0.8686,0.8606,83.2,0.358484


In [292]:
# export df to excel
df.to_excel("cities.xlsx", index=False)